In [4]:
# The Dataset consist of store sales from 2013 to 2017 across 54 different stores and 33 unique product families.
# The task is to predict sales for the nest 15 days.
# >> My idea is to sort the data and do three main predictions
# 1) Total sales across ALL 33 products for Each store,
# 2) Total Sales across ALL 54 sytores for each unique Produc and 
# 3) Sales per store per unique product family. 
# The Code below show the code implemetation for task #1
# NB: Linear regression form sckit works well for all the above three models 


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima  as pmd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import itertools
import sys
import warnings


In [5]:
# *************** Read Data ****************
#
%matplotlib
store_sales = pd.read_csv('train.csv', usecols=['store_nbr', 'family', 'date', 'sales'],
                          dtype = {'store_nbr': 'int8',
                                  'family'    : 'category',
                                  'sales'     : 'float32'},
                          parse_dates=['date'],
                          infer_datetime_format=True,)
#
# Sort the Sales according to Store numbers
#
store_nber = 1          # 1 to 54
DF_store   = store_sales.loc[store_sales.store_nbr == store_nber]
DF_store   = DF_store.groupby(['date']).sales.sum().rename("Total_sales").to_frame()
DF_store.plot(legend = True, title  = 'store_{}'.format(str(store_nber)))
#
# Sort the Last last year (2017) 
#
year = 217        # Represents the most current trend in the data
X    = DF_store.copy()
X.index = X.index.to_period('D')
X['year']  = X.index.year
X          = X.loc[X.year == 2017]
X[['Total_sales']].plot(legend = True, title  = 'store_{}'.format(str(store_nber)))
#
# Split the Data in to test train sets
#
DF_train, DF_test = train_test_split(X, test_size = 0.2, random_state = 42, shuffle = False)
DF_train['Total_sales'].plot(label = 'train set', legend = True, title = "store_{}".format(str(store_nber)))
DF_test['Total_sales'].plot(label = 'test set', legend = True)
DF_test.index

Using matplotlib backend: QtAgg


PeriodIndex(['2017-07-01', '2017-07-02', '2017-07-03', '2017-07-04',
             '2017-07-05', '2017-07-06', '2017-07-07', '2017-07-08',
             '2017-07-09', '2017-07-10', '2017-07-11', '2017-07-12',
             '2017-07-13', '2017-07-14', '2017-07-15', '2017-07-16',
             '2017-07-17', '2017-07-18', '2017-07-19', '2017-07-20',
             '2017-07-21', '2017-07-22', '2017-07-23', '2017-07-24',
             '2017-07-25', '2017-07-26', '2017-07-27', '2017-07-28',
             '2017-07-29', '2017-07-30', '2017-07-31', '2017-08-01',
             '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05',
             '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
             '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
             '2017-08-14', '2017-08-15'],
            dtype='period[D]', name='date')

In [10]:
#************** OPtimal SARIMAX(p,d,q)(P,D,Q,s) ***************
#
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
#
def auto_arima(ts):
    model_order = pmd.auto_arima(ts,
                                 start_p = 1,
                                 start_q = 1,
                                 d = 2,
                                 m       = 7,
                                 seasonal= True,
                                 test    = "adf",
                                 trace   = True,
                                 )
    return model_order

DF_arima_order = auto_arima(DF_train['Total_sales'])
DF_arima_order.summary()

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(1,0,1)[7]             : AIC=inf, Time=0.37 sec
 ARIMA(0,2,0)(0,0,0)[7]             : AIC=3696.148, Time=0.02 sec
 ARIMA(1,2,0)(1,0,0)[7]             : AIC=3470.700, Time=0.08 sec
 ARIMA(0,2,1)(0,0,1)[7]             : AIC=inf, Time=0.12 sec
 ARIMA(1,2,0)(0,0,0)[7]             : AIC=3597.052, Time=0.02 sec
 ARIMA(1,2,0)(2,0,0)[7]             : AIC=3438.536, Time=0.40 sec
 ARIMA(1,2,0)(2,0,1)[7]             : AIC=inf, Time=0.74 sec
 ARIMA(1,2,0)(1,0,1)[7]             : AIC=inf, Time=0.30 sec
 ARIMA(0,2,0)(2,0,0)[7]             : AIC=3514.184, Time=0.05 sec
 ARIMA(2,2,0)(2,0,0)[7]             : AIC=3415.890, Time=0.55 sec
 ARIMA(2,2,0)(1,0,0)[7]             : AIC=3444.814, Time=0.17 sec
 ARIMA(2,2,0)(2,0,1)[7]             : AIC=inf, Time=0.85 sec
 ARIMA(2,2,0)(1,0,1)[7]             : AIC=inf, Time=0.40 sec
 ARIMA(3,2,0)(2,0,0)[7]             : AIC=3373.702, Time=0.73 sec
 ARIMA(3,2,0)(1,0,0)[7]             : AIC=3411.872, Time=0.24 s

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                                 y   No. Observations:                  181
Model:             SARIMAX(4, 2, 0)x(2, 0, 0, 7)   Log Likelihood               -1671.878
Date:                           Sun, 20 Nov 2022   AIC                           3357.757
Time:                                   14:37:50   BIC                           3380.068
Sample:                               01-01-2017   HQIC                          3366.804
                                    - 06-30-2017                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.1164      0.062    -17.967      0.000      -1.238      -0.995
ar.L2         -0.9597      0.090    -10.719      0.000      -1.135      -0.784
ar.L3         -0.7732      0.097     -7.938      0.000      -0.964      -0.582
ar.L4         -0.3124      0.087     -3.587      0.000      -0.483      -0.142
ar.S.L7        0.4185      0.059      7.039      0.000       0.302       0.535
ar.S.L14       0.4521      0.057      7.917      0.000       0.340       0.564
sigma2      7.266e+06   5.05e+05     14.387      0.000    6.28e+06    8.26e+06
===================================================================================
Ljung-Box (L1) (Q):                   2.34   Jarque-Bera (JB):                99.88
Prob(Q):                              0.13   Prob(JB):                         0.00
Heteroskedasticity (H):               0.97   Skew:                             0.35
Prob(H) (two-sided):                  0.92   Kurtosis:                         6.59
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [15]:
# ****************** BEST ARIMA(p,d,q)(P,D,Q)s Via Iterations."Grid Search"***********************
#
warnings.filterwarnings("ignore")             # To signore any warning messages
p = q = range(0,3)
d   = range(0,3)
pdq = list(itertools.product(p,d,q))          # Possible Combinations of ARIMA(p,d,q)
s   = 7                                       # s = Seasonal periodocity 
PDQ = [(x[0], x[1], x[2], s) for x in pdq]    # Seasonal P,D,Q's
print("Length of List PDQ: ", str(len(PDQ)))
print("\nLength of List pdq: ", str(len(pdq)))
#
# Interations for Best (p,d,q)x(P,D,Q)
#
best_aic = np.inf
best_pdq = None
best_seas_PDQ = None

ts_original    = DF_train[['Total_sales']]               # Original time series
ts_differenced = ts_original.diff(2).dropna()
for params in pdq:
    for seas_params in PDQ:
        temp_model = sm.tsa.statespace.SARIMAX(ts_original, 
                                              order = params,
                                              seasonal_order = seas_params,
                                              enforce_stationarity = False,     # for AR, default == True
                                              enforce_invertibility = False,    # for MA default == True
                                              )
        temp_model.initialize_approximate_diffuse()
        
        results = temp_model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_pdq = params
            best_seas_PDQ = seas_params
            
print("SARIMAX {}x{} - AIC: {}".format(best_pdq, best_seas_PDQ, results.aic))        
            

Length of List PDQ:  27

Length of List pdq:  27


KeyboardInterrupt: 

In [102]:
# **************** Train and Test the Model ****************
import error_test
# ARIMA(35,2,0)(2,0,0)[7] 
%matplotlib
#
# Training
#
model = SARIMAX(DF_train['Total_sales'], order =(30,2,0), seas_order = (2,0,0,7))
results = model.fit()
results.summary()
#
# Testing
#
startx = len(DF_train)
endx   = len(DF_train)+len(DF_test) + 30   # ! month forecasting 
Predictions = results.predict(startx, endx, dynamic = True).rename("predicted")
Predictions.to_frame()
#
#plotiing
#
fig, ax = plt.subplots()
DF_test['Total_sales'].plot(ax = ax, label = 'Test set',legend = True, title = 'Store_{}'.format(str(store_nber)))
Predictions[:len(DF_test)].plot(label = 'predicted', legend = True)
Predictions[len(DF_test):].plot(label = 'forecast' , legend = True)
ax.set_xlabel('date')
ax.set_ylabel('Total_sales')
plt.show()
# errors
eval_errors.MAPE(DF_test['Total_sales'], Predictions[:len(DF_test)])

Using matplotlib backend: QtAgg

MAPE: 10.294474778014019


2017-07-01    12021.622182
2017-07-02     5327.192242
2017-07-03    10325.328705
2017-07-04    11791.300069
2017-07-05    13304.998574
                  ...     
2017-09-11     8159.247928
2017-09-12     8514.701805
2017-09-13    10089.576815
2017-09-14     8199.503117
2017-09-15     9621.834709
Freq: D, Name: predicted, Length: 77, dtype: float64

In [104]:
holiday_features = pd.read_csv('Holiday_categorical').set_index('date')
df_train = DF_train[['Total_sales']].copy()
df_test  = DF_test[['Total_sales']].copy()

exog_train = df_train.join(holiday_features).fillna(0.0)
exog_train = exog_train.drop('Total_sales', axis = 1)
exog_test  = df_test.join(holiday_features).fillna(0.0)
exog_test = exog_test.drop('Total_sales', axis = 1)
exog_test

,Primer dia del ano,Traslado Primer dia del ano,Carnaval,Provincializacion de Cotopaxi,Viernes Santo,Dia del Trabajo,Dia de la Madre-1,Dia de la Madre,Batalla de Pichincha,Traslado Batalla de Pichincha,Provincializacion de Imbabura,Primer Grito de Independencia,Traslado Primer Grito de Independencia,Independencia de Guayaquil,Dia de Difuntos,Independencia de Cuenca,Provincializacion de Santo Domingo,Provincializacion Santa Elena
date,,,,,,,,,,,,,,,,,,
2017-07-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
# **************** Train and Test the Model ****************
import error_test
# ARIMA(30,2,0)(2,0,0)[7] 
%matplotlib
#
# Training
#
holiday_features = pd.read_csv('Holiday_categorical')
model = SARIMAX(DF_train['Total_sales'], exog = exog_train, order =(30,2,0), seas_order = (2,0,0,7))
results = model.fit()
results.summary()

#
# Testing
#
startx = len(DF_train)
endx   = len(DF_train)+len(DF_test)-1    # ! month forecasting 
pred   = results.predict(startx, endx, exog = exog_test, dynamic = False)
Predictions = pd.DataFrame(pred.values, index = DF_test.index, columns = ['predicted'])
df = df_test.join(Predictions[['predicted']]).fillna(0.0)

fig, ax1 = plt.subplots()
df.plot(legend = True, ax = ax1)
error_test.MAPE(df['Total_sales'], df['predicted'])
# 
#plotiing
#
# fig, ax = plt.subplots()
# DF_test['Total_sales'].plot(ax = ax, label = 'Test set',legend = True, title = 'Store_{}'.format(str(store_nber)))
# Predictions[:len(DF_test)].plot(ax = ax,label = 'predicted', legend = True)
# # Predictions[len(DF_test):].plot(label = 'forecast' , legend = True)
# ax.set_xlabel('date')
# ax.set_ylabel('Total_sales')
# plt.show()

# error_test.MAPE(DF_test['Total_sales'], Predictions[:len(DF_test)])
# error_test.MSE(DF_test['Total_sales'], Predictions[:len(DF_test)])


Using matplotlib backend: QtAgg

MAPE: 10.2862269746098


In [152]:
# ************** Predictions for ALL stores ****************
#
import eval_errors
%matplotlib
df  = store_sales.loc[store_sales.store_nbr == 1].groupby(['date']).sales.sum().rename('store_1').to_frame()                 
for jj in range(1, 55):
    df['store_{}'.format(jj)] = store_sales.loc[store_sales.store_nbr == jj].groupby(['date']).sales.sum().rename('store_{}'.format(jj)).to_frame()
df['year'] = df.index.year
year = 2017
df   = df.loc[df.year == year]
df   = df.drop('year', axis = 1)
DF_train, DF_test = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = False)
#
# Testing
#
startx = len(DF_train)
endx   = len(DF_train)+len(DF_test)   # ! month forecasting
errors  = pd.DataFrame(columns = ['store_number', 'MAPE'])
for ii in range(54, 55):
    model   = SARIMAX(DF_train['store_{}'.format(str(ii))], order =(30,2,0), seas_order = (2,0,0,7))
    results = model.fit()
    pred    = results.predict(startx, endx, dynamic = True).rename("predicted")
    pred.to_frame()
    df1    = DF_test['store_{}'.format(str(ii))]
    error  = np.mean(np.abs((df1[2:] - pred[2:])/df1[2:]))*100
    errors.loc[ii] = [str(ii), error]
    #
    #Plotting
    #
    fig, ax = plt.subplots()
    DF_test['store_{}'.format(str(ii))].plot(ax= ax, legend = True)
    pred.plot(ax= ax, legend = True)
errors = errors.reset_index().set_index('store_number')
errors = errors.drop('index', axis = 1)
display(errors)

Using matplotlib backend: QtAgg


,MAPE
store_number,
54,17.518568


In [144]:
error